In [1]:
import pandas as pd
import numpy as np
import ast
import gzip
import json
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [2]:
def descompimir_json(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
      for line in archivo.readlines():
          fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)                                                 
    df = df.explode(variable_anidada).reset_index()                         
    df = df.drop(columns="index")                                           
    df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)   
    df = df.drop(columns=variable_anidada)                                  

    return df

In [3]:
user_items = descompimir_json('../Data/users_items.json.gz','items')

In [4]:
user_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


In [5]:
copia_items = user_items.copy()

In [6]:
#Se eliminan las columnas que no se van a utilizar para las funciones solicitadas

user_items=user_items.drop(columns=['user_url','playtime_2weeks','steam_id', 'items_count'])

In [7]:
#Se verifica el tipo de dato de cada columa

user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(1), object(3)
memory usage: 157.8+ MB


In [8]:
#Convertimos la columna item_id en tipo de dato flotante

user_items['item_id'] = user_items['item_id'].astype(float)

In [9]:
user_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10.0,Counter-Strike,6.0
1,76561197970982479,20.0,Team Fortress Classic,0.0
2,76561197970982479,30.0,Day of Defeat,7.0
3,76561197970982479,40.0,Deathmatch Classic,0.0
4,76561197970982479,50.0,Half-Life: Opposing Force,0.0
...,...,...,...,...
5170010,76561198329548331,373330.0,All Is Dust,0.0
5170011,76561198329548331,388490.0,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570.0,You Have 10 Seconds 2,4.0
5170013,76561198329548331,519140.0,Minds Eyes,3.0


In [10]:
#Se normalizan los nombres de los registros que se encuentran en la columna item_name

user_items['item_name'] = user_items['item_name'].str.capitalize()

In [11]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [12]:
user_items.describe()

,item_id,playtime_forever
count,5153209.00,5153209.00
mean,178448.14,991.50
std,131859.78,5418.20
min,10.00,0.00
25%,34460.00,0.00
50%,214420.00,34.00
75%,266430.00,355.00
max,530720.00,642773.00


In [13]:
#Pasamos a horas la columna playtime_forever en donde se puede observar que posee la cantidad de minutos jugados

user_items['playtime_forever'] = user_items['playtime_forever'] / 60

In [14]:
user_items.describe()

,item_id,playtime_forever
count,5153209.00,5153209.00
mean,178448.14,16.52
std,131859.78,90.30
min,10.00,0.00
25%,34460.00,0.00
50%,214420.00,0.57
75%,266430.00,5.92
max,530720.00,10712.88


In [15]:
#Se verifican los datos nulos que poseen las columnas del Dataframe
user_items.isna().sum()

user_id                 0
item_id             16806
item_name           16806
playtime_forever    16806
dtype: int64

In [16]:
#Calculamos el porcentaje de valores nulos que posee cada columna para considerar como proceder

porcentaje_nulos_por_columna = (user_items.isnull().sum() / len(user_items)) * 100
print(porcentaje_nulos_por_columna)


user_id            0.00
item_id            0.33
item_name          0.33
playtime_forever   0.33
dtype: float64


In [17]:
# Al ser los nulos un porcentaje muy bajo, se procede a eliminarlos 

user_items.dropna(inplace=True)

In [18]:
user_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10.00,Counter-strike,0.10
1,76561197970982479,20.00,Team fortress classic,0.00
2,76561197970982479,30.00,Day of defeat,0.12
3,76561197970982479,40.00,Deathmatch classic,0.00
4,76561197970982479,50.00,Half-life: opposing force,0.00
...,...,...,...,...
5170009,76561198329548331,346330.00,Brainbread 2,0.00
5170010,76561198329548331,373330.00,All is dust,0.00
5170011,76561198329548331,388490.00,One way to die: steam edition,0.05
5170012,76561198329548331,521570.00,You have 10 seconds 2,0.07


In [19]:
#Se eliminan los registros que se encuentren duplicados en las columnas item_id y user_id

user_items.drop_duplicates(subset=['item_id','user_id'], inplace=True)

In [21]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094082 entries, 0 to 5170013
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           float64
 2   item_name         object 
 3   playtime_forever  float64
dtypes: float64(2), object(2)
memory usage: 194.3+ MB


In [24]:
#Se mantiene en la columna 'playtime_forever' solo las filas en donde los usuarios hayan jugado más de una hora

user_items = user_items.loc[user_items['playtime_forever'] >= 1]

In [25]:
user_items

,user_id,item_id,item_name,playtime_forever
8,76561197970982479,300.00,Day of defeat: source,78.88
9,76561197970982479,240.00,Counter-strike: source,30.88
10,76561197970982479,3830.00,Psychonauts,5.55
11,76561197970982479,2630.00,Call of duty 2,1.25
12,76561197970982479,3900.00,Sid meier's civilization iv,5.63
...,...,...,...,...
5169481,76561198320038728,442080.00,Riders of icarus,2.55
5169797,76561198320136420,273350.00,Evolve stage 2,1.72
5169804,ArkPlays7,730.00,Counter-strike: global offensive,75.95
5169805,ArkPlays7,346110.00,Ark: survival evolved,10.38


In [26]:
user_items.to_csv('../Data/Data-Limpia/user_items_limpio.csv',index=False)

In [27]:
with gzip.open('../Data/Data-Limpia/user_items_limpio.csv.gz', 'wb') as f:
    user_items.to_csv(f, index=False, encoding='utf-8')